In [3]:
import numpy as np
from scipy  import stats

In [12]:
#Generate full data
def sim(alpha = 0.9, beta = 0.2, 
        gamma = 0.1, lambda0 = 1, lambda1 =5, n = 100, T = 10, C1 = 2):
    #Create CPD Gamma matrix
    matrix_CPD = np.array([
    [1 - gamma, 0, gamma],
    [0, 1 - gamma, gamma],
    [beta / 2, beta / 2, 1 - beta]
])
    #method for translating multinomial to single values    
    translate_val = {(1,0,0): 0, (0,1,0): 1, (0,0,1): 2}

    #CPD for Z_t_i
    z_t_i_CPD = {0: 1-alpha, 1: alpha, 2: 0.5}


    def generate_C(T = T, C1 = C1):
        C_list = [C1]
        for i in range(T-1):
            C_i_CPD = list(matrix_CPD[C_list[i]])
            C = translate_val[tuple(np.random.multinomial(1, pvals = C_i_CPD))]
            C_list.append(C)
        return C_list

    def generate_z_t(Cs, n = n):
        probabilities = [z_t_i_CPD[C_i] for C_i in Cs]
        
        return [np.random.binomial(1, p = prob, size = n) for prob in probabilities]
    
        
    Cs = generate_C(T, C1)
    Zs = generate_z_t(Cs, n = n) 
    Z_flat = np.array(Zs).flat
    Xs = np.reshape([np.random.poisson(lambda0+(lambda1-lambda0)*z) for z in Z_flat], newshape=(T, n))
    return [Cs, Zs, Xs]
    

ls = sim(n = 1000, T = 1000, C1 = 2)
ls



[[2,
  2,
  2,
  2,
  2,
  1,
  1,
  2,
  2,
  2,
  2,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  1,
  1,
  1,
  2,
  1,
  1,
  2,
  2,
  2,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  1,
  1,
  2,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
